# Project 1 Code
### Serena Tang, Liwen, Winston

#### Getting familiar with the dataset

In [4]:
import pandas as pd
classification = pd.read_csv('Project1-Classification.csv')
classification.shape


(3150, 8)

#### Binary Classification

In [1]:
import numpy as np
import random
np.random.seed(42)
random.seed(42)